# Getting Start

此 notebook 负责测试python各个函数的实现情况。

配置日志 & 加载路径 & 检查Setting清单

In [4]:
import sys
from pathlib import Path
import logging
import pandas as pd
from datetime import datetime

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

print("=== 路径设置测试 ===")
print(f"当前工作目录: {Path.cwd()}")
print(f"当前 Python 路径: {sys.path[:3]}...")  # 只显示前3个避免太长

# 正确的路径设置
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"\n项目根目录: {project_root}")

# 检查系统配置
from src.config.settings import config

print("=== 系统配置 ===")
print(f"特征窗口大小: {config.FEATURE_WINDOW_SIZE}")
print(f"时间窗口设置: {config.TIME_WINDOWS}")
print(f"分类阈值类别数: {len(config.CLASSIFICATION_THRESHOLDS)}")
print()

print("分类标准:")
for class_label, (min_val, max_val) in config.CLASSIFICATION_THRESHOLDS.items():
    print(f"  类别 {class_label}: {min_val}% 到 {max_val}%")

=== 路径设置测试 ===
当前工作目录: /Users/anthony/Documents/github/technial_analysis_helper/notebooks
当前 Python 路径: ['/Users/anthony/Documents/github/technial_analysis_helper', '/Users/anthony/Documents/github/technial_analysis_helper', '/usr/local/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python313.zip']...

项目根目录: /Users/anthony/Documents/github/technial_analysis_helper
=== 系统配置 ===
特征窗口大小: 300
时间窗口设置: {'short': 12, 'medium': 48, 'long': 192}
分类阈值类别数: 7

分类标准:
  类别 1: -100% 到 -5.0%
  类别 2: -5.0% 到 -2.0%
  类别 3: -2.0% 到 -0.5%
  类别 4: -0.5% 到 0.5%
  类别 5: 0.5% 到 2.0%
  类别 6: 2.0% 到 5.0%
  类别 7: 5.0% 到 100%


In [6]:
# 测试数据库连接
from src.database.mongo_handler import mongo_handler

print("正在测试 MongoDB 连接...")

try:
    if mongo_handler.connect():
        print("✓ MongoDB 连接成功")
        
        # 检查现有数据量
        collection = mongo_handler.db[config.MONGODB_COLLECTION]
        record_count = collection.count_documents({})
        print(f"数据库中现有记录数: {record_count:,}")
        
        if record_count > 0:
            # 显示最新几条记录的时间范围
            latest_record = collection.find_one(sort=[("timestamp", -1)])
            earliest_record = collection.find_one(sort=[("timestamp", 1)])
            
            if latest_record and earliest_record:
                latest_time = datetime.fromtimestamp(latest_record['timestamp'] / 1000)
                earliest_time = datetime.fromtimestamp(earliest_record['timestamp'] / 1000)
                print(f"数据时间范围: {earliest_time} 到 {latest_time}")
        
        mongo_handler.close()
    else:
        print("✗ MongoDB 连接失败")
except Exception as e:
    print(f"✗ 数据库连接出错: {e}")

2026-01-22 19:46:07,539 - src.database.mongo_handler - INFO - Connected to MongoDB at mongodb://localhost:27017
2026-01-22 19:46:07,539 - src.database.mongo_handler - INFO - Database: technical_analysis
2026-01-22 19:46:07,540 - src.database.mongo_handler - INFO - Collection: candlesticks
2026-01-22 19:46:07,670 - src.database.mongo_handler - INFO - MongoDB connection closed


正在测试 MongoDB 连接...
✓ MongoDB 连接成功
数据库中现有记录数: 53,073
数据时间范围: 2019-12-25 08:00:00 到 2026-01-13 16:00:00


In [ ]:
# 获取最近的市场数据
from src.data.okex_fetcher import okex_fetcher

print("正在获取最新的行情数据...")

try:
    # 获取最近的 K 线数据
    raw_data = okex_fetcher.fetch_candlesticks(bar="1H")
    
    if raw_data:
        # 处理数据格式
        processed_data = okex_fetcher._process_candlestick_data(raw_data)
        
        print(f"✓ 成功获取 {len(processed_data)} 条 K 线数据")
        
        # 显示最新几条数据
        df = pd.DataFrame(processed_data[:5])
        print("\n最新 5 条数据:")
        print(df[['timestamp', 'open', 'high', 'low', 'close', 'volume']].to_string(index=False))
        
        # 转换时间戳为可读格式
        latest_timestamp = processed_data[0]['timestamp']
        readable_time = datetime.fromtimestamp(latest_timestamp / 1000)
        print(f"\n最新数据时间: {readable_time}")
        
    else:
        print("✗ 未能获取到数据")
        
except Exception as e:
    print(f"✗ 获取数据时出错: {e}")

In [ ]:
# 获取历史数据（用于训练）
print("正在获取历史数据用于模型训练...")

# 设置参数 - 可根据需要调整
MAX_RECORDS = 10000  # 获取 10000 条历史记录

try:
    print(f"请求获取最多 {MAX_RECORDS} 条历史记录...")
    
    # 获取历史数据
    historical_data = okex_fetcher.fetch_historical_data(max_records=MAX_RECORDS)
    
    if historical_data:
        print(f"✓ 成功获取 {len(historical_data)} 条历史数据")
        
        # 显示数据统计
        df_hist = pd.DataFrame(historical_data)
        print(f"\n数据统计:")
        print(f"  时间范围: {datetime.fromtimestamp(df_hist['timestamp'].min()/1000)} "
              f"到 {datetime.fromtimestamp(df_hist['timestamp'].max()/1000)}")
        print(f"  价格范围: ${df_hist['close'].min():.2f} 到 ${df_hist['close'].max():.2f}")
        print(f"  平均成交量: {df_hist['volume'].mean():,.0f}")
        
        # 显示价格走势
        print(f"\n价格变化:")
        print(f"  开始价格: ${df_hist['close'].iloc[0]:.2f}")
        print(f"  结束价格: ${df_hist['close'].iloc[-1]:.2f}")
        total_return = (df_hist['close'].iloc[-1] / df_hist['close'].iloc[0] - 1) * 100
        print(f"  总收益率: {total_return:+.2f}%")
        
    else:
        print("✗ 未能获取到历史数据")
        
except Exception as e:
    print(f"✗ 获取历史数据时出错: {e}")

In [ ]:
# 将数据保存到 MongoDB
print("正在将数据保存到 MongoDB...")

if 'historical_data' in locals() and historical_data:
    try:
        # 重新连接数据库
        if mongo_handler.connect():
            print("✓ 数据库连接成功")
            
            # 插入数据
            success = mongo_handler.insert_candlestick_data(historical_data)
            
            if success:
                print(f"✓ 成功保存 {len(historical_data)} 条记录到数据库")
                
                # 验证保存结果
                collection = mongo_handler.db[config.MONGODB_COLLECTION]
                total_records = collection.count_documents({})
                print(f"数据库总记录数: {total_records:,}")
                
                # 显示最新记录
                latest_docs = list(collection.find().sort("timestamp", -1).limit(3))
                print("\n数据库中最新的 3 条记录:")
                for doc in latest_docs:
                    time_str = datetime.fromtimestamp(doc['timestamp'] / 1000)
                    print(f"  {time_str}: ${doc['close']:.2f}")
            else:
                print("✗ 数据保存失败")
                
            mongo_handler.close()
        else:
            print("✗ 无法连接到数据库")
            
    except Exception as e:
        print(f"✗ 保存数据时出错: {e}")
else:
    print("没有可用的历史数据，请先运行获取历史数据的步骤")

In [ ]:
# 从 MongoDB 加载数据
print("正在从 MongoDB 加载数据...")

try:
    if mongo_handler.connect():
        print("✓ 数据库连接成功")
        
        # 加载指定数量的数据
        LOAD_LIMIT = 5000  # 加载 5000 条记录
        db_data = mongo_handler.get_candlestick_data(limit=LOAD_LIMIT)
        
        if db_data:
            print(f"✓ 成功加载 {len(db_data)} 条记录")
            
            # 转换为 DataFrame 进行分析
            df_db = pd.DataFrame(db_data)
            
            print(f"\n数据概览:")
            print(f"  时间范围: {datetime.fromtimestamp(df_db['timestamp'].min()/1000)} "
                  f"到 {datetime.fromtimestamp(df_db['timestamp'].max()/1000)}")
            print(f"  记录数: {len(df_db):,}")
            print(f"  重复记录: {df_db.duplicated(subset=['timestamp']).sum()} 条")
            
            # 显示基本统计信息
            print(f"\n价格统计:")
            print(df_db[['open', 'high', 'low', 'close']].describe())
            
        else:
            print("✗ 数据库中没有数据")
            
        mongo_handler.close()
    else:
        print("✗ 无法连接到数据库")
        
except Exception as e:
    print(f"✗ 加载数据时出错: {e}")

## 总结

本 notebook 完成了以下任务：

1. ✅ **环境设置** - 配置了必要的路径和日志
2. ✅ **系统配置检查** - 验证了项目的配置参数
3. ✅ **数据库连接测试** - 确认 MongoDB 连接正常
4. ✅ **实时数据获取** - 从 OKEx API 获取最新行情数据
5. ✅ **历史数据获取** - 批量获取用于训练的历史数据
6. ✅ **数据持久化** - 将获取的数据保存到 MongoDB
7. ✅ **数据加载验证** - 从数据库重新加载数据进行验证

### 下一步建议

运行完此 notebook 后，可以继续执行：
- `02_feature_engineering.ipynb` - 进行特征工程处理
- `03_model_training.ipynb` - 使用生成的数据训练模型
- `04_prediction.ipynb` - 进行实时价格预测

### 注意事项

- 确保 MongoDB 服务正在运行
- 根据网络情况调整获取数据的数量和频率
- 定期清理重复数据以保持数据库效率